In [76]:
'''
create sql dataset from hdf5 files directly
   
@author - Ken Lee
 this is based on Andrew's id_title_artist.py and Alen's create_song_sql.py
'''

# in this cell, some of the features of MSD dataset are imported and stored in SQL table. 


import hdf5_getters
from tqdm import tqdm
import pymysql.cursors
import pandas as pd
import numpy as np
import glob
import csv
import json
from pprint import pprint
import unicodedata


glob_path = '~/Users/kenleejr92/MillionSongSubset/data/*/*/*/*'
filepaths = glob.glob(glob_path)

# establish connection to sql server
connection = pymysql.connect(host='localhost',\
   user='root',password='yangspike92',db='songs')
cursor = connection.cursor()

# create sql table (only need to do this once)
sql = '''DROP TABLE IF EXISTS songs;'''
cursor.execute(sql)
connection.commit()
sql = '''CREATE TABLE IF NOT EXISTS songs (

songID VARCHAR(50) PRIMARY KEY, 
danceability REAL DEFAULT NULL,
duration REAL DEFAULT NULL,
energy REAL DEFAULT NULL,
loudness REAL DEFAULT NULL,
musicalKey INT DEFAULT NULL,
mode INT DEFAULT NULL,
tempo REAL DEFAULT NULL,
time_signature INT DEFAULT NULL,
year INT DEFAULT NULL,
song_hottness REAL DEFAULT NULL,
max_loudness REAL DEFAULT NULL,
end_of_fade_in REAL DEFAULT NULL,
start_of_fade_out REAL DEFAULT NULL,
bars_start REAL DEFAULT NULL,
beats_start REAL DEFAULT NULL,
sections_start REAL DEFAULT NULL,
tatums_start REAL DEFAULT NULL,
segments_start REAL DEFAULT NULL,
max_loudness_time REAL DEFAULT NULL,
segments_loudness_start REAL DEFAULT NULL,
segments_pitches REAL DEFAULT NULL,
segments_timbre REAL DEFAULT NULL,

INDEX songID (songID)
);'''
cursor.execute(sql)
connection.commit()

glob_path = '/Users/kenleejr92/MillionSongSubset/data/*/*/*/*'
filepaths = glob.glob(glob_path)
for filepath in tqdm(filepaths):
    h5 = hdf5_getters.open_h5_file_read(filepath)
    n = hdf5_getters.get_num_songs(h5)
    for row in range(n):
        song_id = hdf5_getters.get_song_id(h5,songidx=row).decode('UTF-8')
#         artist = hdf5_getters.get_artist_name(h5,songidx=row).decode('UTF-8')
#         title= hdf5_getters.get_title(h5,songidx=row)#.decode('UTF-8')
#         artist = "".join(c for c in unicodedata.normalize('NFD', str(artist.decode("utf8"))) if unicodedata.category(c) != "Mn")
#         title = "".join(c for c in unicodedata.normalize('NFD', str(title.decode("utf8"))) if unicodedata.category(c) != "Mn")
        #single number features
        danceability = hdf5_getters.get_danceability(h5,songidx=row)
        duration = hdf5_getters.get_duration(h5,songidx=row)
        energy = hdf5_getters.get_energy(h5,songidx=row)
        loudness = hdf5_getters.get_loudness(h5,songidx=row)
        musicalKey = hdf5_getters.get_key(h5,songidx=row)
        mode = hdf5_getters.get_mode(h5,songidx=row)
        tempo = hdf5_getters.get_tempo(h5,songidx=row)
        time_signature = hdf5_getters.get_time_signature(h5,songidx=row)
        year = hdf5_getters.get_year(h5,songidx=row)
        song_hottness = hdf5_getters.get_song_hotttnesss(h5,songidx=row)
        end_of_fade_in = hdf5_getters.get_end_of_fade_in(h5,songidx=row)
        start_of_fade_out = hdf5_getters.get_start_of_fade_out(h5,songidx=row)
        
        #timestamp features
        #take last element and divide by length to get beats/unit time, segments/unit_time
        bars_start = hdf5_getters.get_bars_start(h5,songidx=row)
        beats_start = hdf5_getters.get_beats_start(h5,songidx=row)
        sections_start = hdf5_getters.get_sections_start(h5,songidx=row)
        tatums_start = hdf5_getters.get_tatums_start(h5,songidx=row)
        segments_start = hdf5_getters.get_segments_start(h5,songidx=row)
        if len(bars_start)==0: bars_start = 0.
        else: bars_start = bars_start[-1]/len(bars_start)
        if len(beats_start)==0: beats_start = 0.
        else: beats_start = beats_start[-1]/len(beats_start)
        if len(sections_start)==0: sections_start = 0.
        else: sections_start = sections_start[-1]/len(sections_start)
        if len(tatums_start)==0: tatums_start = 0.
        else: tatums_start = tatums_start[-1]/len(tatums_start)
        if len(segments_start)==0: segments_start = 0.
        else: segments_start = segments_start[-1]/len(segments_start)
        
        #time series features
        #take mean
        max_loudness_time = hdf5_getters.get_segments_loudness_max_time(h5,songidx=row)
        segments_loudness_start = hdf5_getters.get_segments_loudness_start(h5,songidx=row)
        segments_pitches = hdf5_getters.get_segments_pitches(h5,songidx=row)
        segments_timbre = hdf5_getters.get_segments_timbre(h5,songidx=row)
        max_loudness = hdf5_getters.get_segments_loudness_max(h5,songidx=row)
        segments_pitches = np.mean(segments_pitches)
        segments_timbre = np.mean(segments_timbre)
        max_loudness = np.mean(max_loudness)
        max_loudness_time = np.mean(max_loudness_time)
        segments_loudness_start = np.mean(segments_loudness_start)
        
        l = [song_id, danceability, duration, energy, loudness, musicalKey, mode, 
             tempo, time_signature, year, song_hottness, max_loudness, end_of_fade_in,
             start_of_fade_out, bars_start, beats_start, sections_start, tatums_start,
             segments_start, max_loudness_time, segments_loudness_start, segments_pitches,
             segments_timbre]
        
        for idx,val in enumerate(l): 
            if type(val)==np.float64 and np.isnan(val):
                l[idx]=0.
                
        query = "INSERT INTO songs (songID, danceability, duration, energy, loudness, musicalKey,\
            mode, tempo, time_signature, year, song_hottness, max_loudness, end_of_fade_in, start_of_fade_out,\
            bars_start, beats_start, sections_start, tatums_start, segments_start, max_loudness_time,\
            segments_loudness_start, segments_pitches, segments_timbre) \
            VALUES ('%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s','%s',\
            '%s','%s','%s','%s','%s','%s','%s','%s','%s','%s')"\
            % tuple(l)
        cursor.execute(query)
        connection.commit()
    h5.close()

# close sql connection
cursor.close()
connection.close()










  0%|          | 0/10000 [00:00<?, ?it/s]










DataError: (1265, u"Data truncated for column 'song_hottness' at row 1")

In [1]:
# in this cell, we store the smaller version of the dataset in a pandas.DataFrame 
# just to get more familiar with the data

import hdf5_getters
from tqdm import tqdm
import pymysql.cursors
import pandas as pd
import numpy as np
import glob
import csv
import pdb
import json
from pprint import pprint
import unicodedata

myList = []
glob_path = '/home/frank/1CSEM/1UTCoursesTaken/dataMiningEE380L/termProject/dataset/MillionSongSubset/data/*/*/*/*'
filepaths = glob.glob(glob_path)

# i = 0
for filepath in tqdm(filepaths):
#     i += 1 
#     if (i == 100):
#         break
    h5 = hdf5_getters.open_h5_file_read(filepath)
    n = hdf5_getters.get_num_songs(h5)
    # print n
    for row in range(n):
        artist = hdf5_getters.get_artist_name(h5,songidx=row)#.decode('UTF-8')
        song_id = hdf5_getters.get_song_id(h5,songidx=row).decode('UTF-8')
        title= hdf5_getters.get_title(h5,songidx=row)#.decode('UTF-8')
        artist = "".join(c for c in unicodedata.normalize('NFD', str(artist.decode("utf8"))) if unicodedata.category(c) != "Mn")
        title = "".join(c for c in unicodedata.normalize('NFD', str(title.decode("utf8"))) if unicodedata.category(c) != "Mn")
        danceability = hdf5_getters.get_danceability(h5,songidx=row)
        duration = hdf5_getters.get_duration(h5,songidx=row)
        energy = hdf5_getters.get_energy(h5,songidx=row)
        loudness = hdf5_getters.get_loudness(h5,songidx=row)
        musicalKey = hdf5_getters.get_key(h5,songidx=row)
        mode = hdf5_getters.get_mode(h5,songidx=row)
        tempo = hdf5_getters.get_tempo(h5,songidx=row)
        artist_mbtags = hdf5_getters.get_artist_mbtags(h5,songidx=row)
        artist_mbtags_count = hdf5_getters.get_artist_mbtags_count(h5,songidx=row)
        artist_terms = hdf5_getters.get_artist_terms(h5, songidx=row)
        artist_mbid = hdf5_getters.get_artist_mbid(h5, songidx=row).decode('UTF-8')
        tempo = hdf5_getters.get_tempo(h5,songidx=row)
        row = [artist, artist_mbid, song_id,artist_terms, artist_mbtags]
        myList.append(row)
    h5.close()
        # print (artist)
        # print (title)
        # print (song_id)
        # print (artist_terms)
        # print(tempo)
        # print(artist_mbtags)
        # print(artist_mbtags_count)
        # pdb.set_trace()
        # populate sql table with data
df = pd.DataFrame(myList, columns=['artist_name','artist_mbid','song_id','artist_terms','artist_mbtags'])
df.head(100)
df.loc[df['artist_name'] == 'The Killers']

100%|██████████| 10000/10000 [02:46<00:00, 60.13it/s]


,artist_name,artist_mbid,song_id,artist_terms,artist_mbtags
85,The Killers,95e1ead9-4d31-4808-a7ac-32c3614c116b,SOTEFFR12A8C144765,"[b'rock', b'alternative', b'rave', b'disco', b...","[b'rock and indie', b'united states', b'altern..."
245,The Killers,95e1ead9-4d31-4808-a7ac-32c3614c116b,SORGIYZ12AAF3B3528,"[b'rock', b'alternative', b'rave', b'disco', b...","[b'rock and indie', b'united states', b'altern..."
4617,The Killers,95e1ead9-4d31-4808-a7ac-32c3614c116b,SOSNQQX12AF729C44C,"[b'rock', b'alternative', b'rave', b'disco', b...","[b'rock and indie', b'united states', b'altern..."
4798,The Killers,95e1ead9-4d31-4808-a7ac-32c3614c116b,SOQFWBE12A8C1316AF,"[b'rock', b'alternative', b'rave', b'disco', b...","[b'rock and indie', b'united states', b'altern..."
5232,The Killers,95e1ead9-4d31-4808-a7ac-32c3614c116b,SOQNLMA12A8C1414FC,"[b'rock', b'alternative', b'rave', b'disco', b...","[b'rock and indie', b'united states', b'altern..."
9375,The Killers,95e1ead9-4d31-4808-a7ac-32c3614c116b,SOGOQGE12AB0182907,"[b'rock', b'alternative', b'rave', b'disco', b...","[b'rock and indie', b'united states', b'altern..."
